<a href="https://colab.research.google.com/github/B0gdanovAleksandr/Neuro-Quality-Control/blob/main/%22%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%BA%D0%BE%D0%BD%D1%82%D1%80%D0%BE%D0%BB%D1%8C_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D0%9E%D0%9F%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка необходимых библиотек
!pip install gradio_client
!pip install sqlalchemy
!pip install openai

import os
import sqlite3
import pandas as pd
from google.colab import drive
from gradio_client import Client, handle_file
import openai
print(openai.__version__)
from openai import OpenAI


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 9.7 MB/s eta 0:00:00
1.75.0


In [ ]:
#  Подключение к Google Диску
#  открытая ссылка для подключения к папке на гугл диске для экспериментов: https://drive.google.com/drive/folders/1pCFNqXPrXsPv3yRkpmZDSAfzvFYO51bW?usp=sharing

drive.mount('/content/drive')
google_drive_path = '/content/drive/My Drive/recordings/' #  вставьте ссылку на вашу папку гугл диска, куда сохраняете записи


Mounted at /content/drive


In [ ]:
#  Функция для транскрибирования аудиофайлов
def transcribe_audio(file_path):
    client = Client("hf-audio/whisper-large-v3-turbo")
    result = client.predict(
        inputs=handle_file(file_path),
        task="transcribe",
        api_name="/predict"
    )
    return result

In [ ]:
#  Создание базы данных SQLite
db_path = '/content/transcriptions.db'
conn = sqlite3.connect(db_path)
c = conn.cursor()

# Создание таблицы для хранения транскрипций
c.execute('''
CREATE TABLE IF NOT EXISTS transcriptions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    filename TEXT,
    transcription TEXT
)
''')
conn.commit()

In [ ]:
# Подключение к Google Диску
drive.mount('/content/drive')

# Укажите путь к вашей папке
google_drive_path = '/content/drive/My Drive/recordings/'

# Проверка наличия папки
if os.path.exists(google_drive_path):
    print("Папка найдена.")
    files = os.listdir(google_drive_path)
    print("Файлы в папке:", files)
else:
    print("Папка не найдена. Пожалуйста, проверьте путь.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Папка найдена.
Файлы в папке: ['8.mp3', '10.mp3', '6.mp3', '5.mp3', '2.mp3', '4.mp3', '7.mp3', '9.mp3', '3.mp3', '1.mp3']


In [ ]:

#  Процесс обработки аудиофайлов
for filename in os.listdir(google_drive_path):
    if filename.endswith('.mp3'):  # Можно изменить на нужный формат
        full_path = os.path.join(google_drive_path, filename)
        print(f'Transcribing {full_path}...')

        # Транскрибируем аудиофайл
        transcription = transcribe_audio(full_path)

        # Сохраняем результаты в базу данных
        c.execute("INSERT INTO transcriptions (filename, transcription) VALUES (?, ?)", (filename, transcription))
        conn.commit()

Transcribing /content/drive/My Drive/recordings/8.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /content/drive/My Drive/recordings/10.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /content/drive/My Drive/recordings/6.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /content/drive/My Drive/recordings/5.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /content/drive/My Drive/recordings/2.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /content/drive/My Drive/recordings/4.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /content/drive/My Drive/recordings/7.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /content/drive/My Drive/recordings/9.mp3...
Loaded as API: https://hf-audio-whisper-large-v3-turbo.hf.space ✔
Transcribing /c

In [ ]:
#  Извлечение результатов из базы данных и вывод
df = pd.read_sql_query("SELECT * FROM transcriptions", conn)
print(df)

   id filename                                      transcription
0   1    8.mp3   Алло, здравствуйте. День добрый. Это Глеб. Уд...
1   2   10.mp3   Алло. Алло. Екатерина. Да. Да. Общались. Да, ...
2   3    6.mp3   Алло. Алло, здравствуйте. Да, доброе утро. Да...
3   4    5.mp3   Алло. Алло. Добрый день. Да, здравствуйте. Эт...
4   5    2.mp3   Томатологи, здравствуйте. День добрый, а это ...
5   6    4.mp3   Здравствуйте. Здравствуйте. Меня зовут Дмитри...
6   7    7.mp3   Алло. Алло, Наталья, здравствуйте. Да, добрый...
7   8    9.mp3   Алло. Алло, Елена, добрый день. Добрый. Это В...
8   9    3.mp3   Алло, Дмитрий, здравствуйте. Да. Да, меня тож...
9  10    1.mp3   Алло. Халло, здравствуйте. Добрый день. Это к...


In [ ]:
# Убедитесь, что у вас есть действительный ключ API OpenAI
api_key = os.environ.get('OPENAI_API_KEY', 'ВСТАВЬТЕ ВАШ АПИ КЛЮЧ ')  # Убедитесь, что ваш ключ установлен
client = OpenAI(api_key=api_key)

# Функция для анализа звонков (с учетом коротких разговоров)
def analyze_call_general(transcription, is_short=False):
    if is_short:
        prompt = f"""
        Задача: Вы — эксперт по анализу телефонных разговоров.
        Проанализируйте данный короткий разговор и составьте краткое саммари, охватывающее ключевые моменты общения.

        Транскрипт разговора:
        {transcription}

        Формат ответа:
        - Краткое саммари ключевых моментов разговора.
        """
    else:
        prompt = f"""
        Задача: Вы — эксперт по оценке качества продающих звонков.
        Проанализируйте транскрипт разговора менеджера с клиентом и дайте оценки по каждому этапу по шкале от 0 до 2.
        Обязательно подметьте, что было сделано хорошо, и дайте рекомендации по улучшению с конкретными примерами вопросов и формулировок, которые менеджер может использовать.

        Транскрипт разговора:
        {transcription}

        Критерии оценки:
        1. Приветствие и самопрезентация: Менеджер представился и установил позитивный контакт? Оценка: 0 / 1 / 2
        2. Выявление потребностей: Менеджер задавал открытые и уточняющие вопросы? Оценка: 0 / 1 / 2
        3. Презентация под потребности: Менеджер адаптировал описание продукта/услуги? Оценка: 0 / 1 / 2
        4. Отработка возражений: Менеджер выявил истинные возражения клиента? Оценка: 0 / 1 / 2
        5. Закрытие сделки: Менеджер договорился о следующем шаге? Оценка: 0 / 1 / 2

        Формат ответа:
        - Общая оценка (по шкале от 0 до 10)
        - Подробная оценка по каждому критерию с примерами
            1. Пункт: Оценка
                - Положительные моменты
                - Рекомендации по улучшению
                    - Примеры вопросов и формулировок для использования
        - Общая рекомендация, включая конкретные шаги и формулировки для улучшения
        """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [ ]:
# Создание списка для хранения результатов нового анализа
new_analysis_results = []

# Предположим, что ваш код для анализа транскрипций уже выполнен, и вы добавляете результаты в new_analysis_results
for index, row in df.iterrows():
    transcription = row['transcription']

    # Проверка длины транскрипции
    word_count = len(transcription.split())
    print(f"Звонок {index}: количество слов = {word_count}")  # Отладочная информация

    # Условие для определения, является ли разговор коротким (например, длина транскрипции меньше 400 слов)
    is_short = word_count < 400

    # Используем новую функцию анализа
    new_result = analyze_call_general(transcription, is_short=is_short)

    # Добавляем результат в список
    new_analysis_results.append((transcription, new_result))

# Создание DataFrame из результатов нового анализа
analysis_df = pd.DataFrame(new_analysis_results, columns=['transcription', 'analysis'])

# Выводим результаты в формате таблицы
display(analysis_df)

# Сохранение результатов в Excel файл
analysis_df.to_excel('call_analysis_results_new.xlsx', index=False)

Звонок 0: количество слов = 1454
Звонок 1: количество слов = 1895
Звонок 2: количество слов = 1361
Звонок 3: количество слов = 500
Звонок 4: количество слов = 132
Звонок 5: количество слов = 1284
Звонок 6: количество слов = 330
Звонок 7: количество слов = 2425
Звонок 8: количество слов = 769
Звонок 9: количество слов = 380


,transcription,analysis
0,"Алло, здравствуйте. День добрый. Это Глеб. Уд...",Общая оценка: 7/10\n\n1. Приветствие и самопре...
1,"Алло. Алло. Екатерина. Да. Да. Общались. Да, ...",- Общая оценка: 6/10\n\n1. Приветствие и самоп...
2,"Алло. Алло, здравствуйте. Да, доброе утро. Да...",Общая оценка: 6/10\n\n1. Приветствие и самопре...
3,"Алло. Алло. Добрый день. Да, здравствуйте. Эт...",- Общая оценка: 5 из 10\n\n1. Приветствие и са...
4,"Томатологи, здравствуйте. День добрый, а это ...",Саммари: Глеб Бигдата интересуется привлечение...
5,Здравствуйте. Здравствуйте. Меня зовут Дмитри...,- Общая оценка: 6\n\n1. Приветствие и самопрез...
6,"Алло. Алло, Наталья, здравствуйте. Да, добрый...",Саммари: \n- Азер из компании Big Data звонит ...
7,"Алло. Алло, Елена, добрый день. Добрый. Это В...",Общая оценка: 6/10\n\n1. Приветствие и самопре...
8,"Алло, Дмитрий, здравствуйте. Да. Да, меня тож...",Общая оценка: 5/10\n\n1. Приветствие и самопре...
9,"Алло. Халло, здравствуйте. Добрый день. Это к...",В разговоре идет речь о привлечении клиентов л...


In [ ]:
# Установить соединение с базой данных
conn = sqlite3.connect('your_database.db')
c = conn.cursor()

# Создание таблицы для хранения результатов анализа
c.execute('''
CREATE TABLE IF NOT EXISTS call_analysis (
    id INTEGER PRIMARY KEY,
    transcription TEXT,
    analysis TEXT
)
''')

In [ ]:
# Сохранение анализа
for index, row in df.iterrows():
    transcription = row['transcription']
    result = analyze_call_general(transcription, is_short=word_count < 400)  # Получаем результат анализа
    c.execute("INSERT INTO call_analysis (transcription, analysis) VALUES (?, ?)", (transcription, result))

conn.commit()

In [ ]:
# Добавим возможность выгружать результат в Эксель файл.
conn = sqlite3.connect('your_database.db')  # замените на ваш путь к базе данных

# Загружаем результаты из базы данных в DataFrame
df = pd.read_sql_query("SELECT * FROM call_analysis", conn)

# Сохраняем в Excel файл
df.to_excel('call_analysis_results.xlsx', index=False)  # Убедитесь, что указали имя файла

# Закрываем соединение с базой данных
conn.close()